In [9]:
from backend.dsm2_reader import read_scenario_dir
import pandas as pd
import altair as alt
from data_config import gatef, elev_list, flow_list, stn_name
import numpy as np
import os
from pandas import DataFrame
import pickle

In [10]:
stn_list = ['MID_GATEOP','GLC_GATEOP','OLD_GATEOP']

In [11]:
data = read_scenario_dir("C:/Users/Inigo/Projects/sdg-dashboard/data", v7_filter="_7")

mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_SDG_7.dss
mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV2Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV2Ma_SDG_7.dss
DSS file opened successfully.
DSS file opened successfully.
DSS file opened successfully.
DSS file opened successfully.


In [16]:
print(data.keys())

dict_keys(['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss'])


In [17]:
def calc_vel(flow,stage_up, bottom_elev, width):
    #velocity is flow/cross-section
    
    xs = (stage_up-bottom_elev)*width
    vel = flow/xs
    return vel

def filter_data(data, column, values, start_date=None, end_date=None):
    """Filter data based on a column and a list of values."""
    if start_date:
        data = data[data[column].isin(values)].dropna()
        data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]
    else:
        data = data[data[column].isin(values)].dropna()
    return data

def rename_gates(data, column, rename_map):
    """Rename values in a column in the DataFrame."""
    data[column] = data[column].replace(rename_map)
    return data

def set_datetime_index(data):
    data.set_index('datetime', inplace=True)
    data = data['value']
    return(data)

def parse_dss_filename(file_path: str) -> str:
    """
    Extract the last value from a DSS file path, excluding the extension.

    Parameters:
    - file_path (str): Full path to the DSS file.

    Returns:
    - str: The model name extracted from DSS file.
    """
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    model_name = file_name.split('_')[0]
    return model_name

# def filter_date(data, start_date, end_date):
#     return data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]

# def get_gate_up(data, column)

def prepare_full_data(sdg_flow, sdg_stage, sdg_gateop, hydro_wl, gatef, model):
    """
    Prepare gate data dictionary from input datasets.
    
    Parameters:
    - sdg_flow: DataFrame with flow data.
    - sdg_stage: DataFrame with stage data.
    - gatop: DataFrame with gate operation data.
    - hydro_wl: Dataframe with water level data.
    - gatef: DataFrame containing gate configuration.

    Returns:
    - Dictionary containing gate data.
    """
    full_data = {}
    for i, gate_name in enumerate(gatef['name']):
        flow_data = filter_data(sdg_flow, 'gate_op', [gatef["flow_op"][i]])
        gate_data = filter_data(sdg_stage, 'gate_op', [gatef['gate_status'][i]])
        gate_data = gate_data[gate_data['unit'] == "FEET"]
        gateop_data = sdg_gateop[sdg_gateop['gate_op']==gatef['station'][i]]
        hydro_wl_data = hydro_wl[hydro_wl['gate']==gatef['station'][i]]
        full_data[gatef["ID"][i]] = {
            'name': gatef['name'][i],
            'bottom_elev': gatef['bottom_elev'][i],
            'width': gatef['width'][i],
            'flow_data': flow_data, #sc2_flow
            'gate_data': gate_data, #sc2_gate
            'gate_operation_data': gateop_data, #sc2_gateop
            "water_level_data": hydro_wl_data, #sc2_wl,
            "model": model
            
        }
    return full_data

def generate_full_model_data(data, path, gatef, elev_list, flow_list, stn_name, stn_list, start_date=None, end_date=None):
    sdg = data[path]['sdg']
    hydro = data[path]['hydro']
    model = parse_dss_filename(path)

    sdg_stage = filter_data(sdg, 'gate_op', elev_list, start_date, end_date)
    sdg_flow = filter_data(sdg, 'gate_op', flow_list, start_date, end_date)
    sdg_gateop = filter_data(sdg, 'gate_op', stn_list, start_date, end_date)
    gate_names = {'MID_GATEOP': 'MHO', 'GLC_GATEOP': 'DGL', 'OLD_GATEOP': 'OLD'}
    sdg_gateop = rename_gates(sdg_gateop, "gate_op", gate_names)
    
    hydro_wl = hydro[hydro['parameter']=="STAGE"]
    hydro_wl = filter_data(hydro_wl, 'gate', stn_name, start_date, end_date)

    full_data = prepare_full_data(sdg_flow, sdg_stage, sdg_gateop, hydro_wl, gatef, model)
    
    sdg_flow_GLC_FLOW_FISH = set_datetime_index(full_data['GLC']['flow_data'])
    sdg_flow_GLC_GATE_UP = set_datetime_index(full_data['GLC']['gate_data'])
    glc_bottom_elev = full_data['GLC']['bottom_elev']
    glc_width = full_data['GLC']['width']


    sdg_flow_MID_FLOW_FISH = set_datetime_index(full_data['MID']['flow_data'])
    sdg_flow_MID_GATE_UP = set_datetime_index(full_data['MID']['gate_data'])
    mid_bottom_elev = full_data['MID']['bottom_elev']
    mid_width = full_data['MID']['width']
    
    sdg_flow_OLD_FLOW_FISH = set_datetime_index(full_data['OLD']['flow_data'])
    sdg_flow_OLD_GATE_UP = set_datetime_index(full_data['OLD']['gate_data'])
    old_bottom_elev = full_data['OLD']['bottom_elev']
    old_width = full_data['OLD']['width']
    
    full_data["GLC"]['vel'] = pd.DataFrame(calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width))
    full_data["MID"]['vel'] = pd.DataFrame(calc_vel(sdg_flow_MID_FLOW_FISH,sdg_flow_MID_GATE_UP,mid_bottom_elev, mid_width))
    full_data["OLD"]['vel'] = pd.DataFrame(calc_vel(sdg_flow_OLD_FLOW_FISH,sdg_flow_OLD_GATE_UP,old_bottom_elev, old_width))

    for key in ["GLC", "MID", "OLD"]:
        full_data[key]['vel']['datetime'] = full_data[key]['vel'].index
        full_data[key]['vel'] = full_data[key]['vel'].reset_index(drop=True)
        full_data[key]['vel'] = full_data[key]['vel'][["datetime", "value"]]
    
    return full_data

def generate_multi_model_data(data, paths, gatef, elev_list, flow_list, stn_name, stn_list, start_date=None, end_date=None):
    """
    Generate a dictionary containing data for multiple models.
    
    Parameters:
    - data: Dictionary of datasets keyed by file paths.
    - paths: List of file paths to process.
    - gatef: DataFrame with gate configurations.
    - elev_list, flow_list, stn_name, stn_list: Parameters for filtering data.
    - start_date, end_date: Optional date range for filtering.

    Returns:
    - Dictionary containing data for all models.
    """
    all_model_data = {}
    for path in paths:
        # Extract data for a single model
        model_data = generate_full_model_data(data, path, gatef, elev_list, flow_list, stn_name, stn_list, start_date, end_date)
        model_name = parse_dss_filename(path)  # Extract the model name
        
        # # Store the data keyed by the model name
        all_model_data[model_name] = model_data
    
    return all_model_data

In [365]:
# list = ['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']
# for path in list:
#     print(path)
start_zoom = '2016-05-01'
end_zoom = '2016-11-30'
multi_model_data = generate_multi_model_data(
    data,
    ['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss'],
    gatef,
    elev_list,
    flow_list,
    stn_name,
    stn_list,
    start_zoom,
    end_zoom)


C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_hydro_7.dss
C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_hydro_7.dss


In [ ]:
print(multi_model_data['FPV1Ma'][])

{'GLC': {'name': 'GrantLine', 'bottom_elev': -6, 'width': 5, 'flow_data':                           value unit        gate_op scenario
datetime                                                    
2016-05-01 00:00:00  -23.797087  CFS  GLC_FLOW_FISH   FPV1MA
2016-05-01 00:15:00  -27.210817  CFS  GLC_FLOW_FISH   FPV1MA
2016-05-01 00:30:00  -28.053200  CFS  GLC_FLOW_FISH   FPV1MA
2016-05-01 00:45:00  -29.390024  CFS  GLC_FLOW_FISH   FPV1MA
2016-05-01 01:00:00  -29.655567  CFS  GLC_FLOW_FISH   FPV1MA
...                         ...  ...            ...      ...
2016-11-29 23:00:00  646.536743  CFS  GLC_FLOW_FISH   FPV1MA
2016-11-29 23:15:00  665.150452  CFS  GLC_FLOW_FISH   FPV1MA
2016-11-29 23:30:00  685.314209  CFS  GLC_FLOW_FISH   FPV1MA
2016-11-29 23:45:00  700.422058  CFS  GLC_FLOW_FISH   FPV1MA
2016-11-30 00:00:00  712.635681  CFS  GLC_FLOW_FISH   FPV1MA

[20449 rows x 4 columns], 'gate_data':                         value  unit      gate_op scenario
datetime                           

In [46]:
def post_process_gateop(multi_model_data, model, gate, year=None, start_date=None, end_date=None):
    gate_data = multi_model_data[model][gate]['gate_operation_data']
    if year:
        gate_data["year"] = gate_data["datetime"].dt.year
        gate_data = gate_data[gate_data["year"] == year]
    # if start_date:
    #     gate_data['date'] = gate_data["datetime"].dt.date
    #     gate_data = gate_data[(gate_data["date"] >= start_date) & 
    #                           (gate_data["date"] <= end_date)]
    gate_data = gate_data.loc[
            gate_data['datetime'].dt.month.between(5, 11)
    ]
        # gate_data['datetime'].dt.month.between(5, 11)
    # ]
    gate_up_df = gate_data[["datetime", "value"]]
    gate_up_df["gate_status"] = gate_up_df['value']>=10
    gate_up_df['consecutive_groups'] = (gate_up_df['value'] != gate_up_df['value'].shift()).cumsum()
    gate_up_df['min_datetime'] = gate_up_df.groupby('consecutive_groups')['datetime'].transform('min')
    gate_up_df['max_datetime'] = gate_up_df.groupby('consecutive_groups')['datetime'].transform('max')
    consecutive_streaks = gate_up_df.groupby(['consecutive_groups', 'value', 'min_datetime', 'max_datetime']).size().reset_index(name='count')
    consecutive_streaks['streak_duration'] = consecutive_streaks['count'] * 15 / 60
    consecutive_streaks_clean = consecutive_streaks.drop(['value', 'consecutive_groups', 'max_datetime'], axis = 1)
    # print(consecutive_streaks_clean.head())
    merged_gate_df = pd.merge(gate_up_df, consecutive_streaks_clean,left_on="min_datetime", right_on="min_datetime")
    merged_gate_df = merged_gate_df.drop(['consecutive_groups', 'value'], axis=1)
    merged_gate_df = merged_gate_df.rename(columns={"min_datetime": "gate_min_datetime", 
                                                "max_datetime": "gate_max_datetime",
                                                "count": "gate_count",
                                                "streak_duration": "gate_streak_duration"})
    return merged_gate_df

def post_process_velocity(multi_model_data, model, gate, year=None, start_date=None, end_date=None):
    vel_zoom_df =multi_model_data[model][gate]["vel"]
    if year:
        vel_zoom_df["year"] = vel_zoom_df["datetime"].dt.year
        vel_zoom_df = vel_zoom_df[vel_zoom_df["year"] == year]
    vel_zoom_df = vel_zoom_df.loc[
            vel_zoom_df['datetime'].dt.month.between(5, 11)
    ]
    # if start_date:
    #     vel_zoom_df['date'] = vel_zoom_df["datetime"].dt.date
    #     vel_zoom_df = vel_zoom_df[(vel_zoom_df["date"] >= start_date) & 
    #                               (vel_zoom_df["date"] <= end_date)]
    vel_zoom_df['Velocity_Category'] = np.where(vel_zoom_df['value'] >= 8, "Over 8ft/s", "Under 8ft/s")
    #.shift shift value down and compare each value with the previous row; increase value when rows are different
    vel_zoom_df['consecutive_groups'] = (vel_zoom_df['Velocity_Category'] != vel_zoom_df['Velocity_Category'].shift()).cumsum()
    vel_zoom_df['min_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('min')
    vel_zoom_df['max_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('max')
    vel_zoom_df['date'] = vel_zoom_df['datetime'].dt.date.astype(str)
    consecutive_streaks_vel = vel_zoom_df.groupby(['consecutive_groups', 'Velocity_Category', 'min_datetime', 'max_datetime']).size().reset_index(name='count')
    consecutive_streaks_vel['streak_duration'] = consecutive_streaks_vel['count'] * 15 / 60

    consecutive_streaks_vel_clean = consecutive_streaks_vel.drop(['consecutive_groups', 'Velocity_Category', 'max_datetime'], axis=1)
    merged_vel_df = pd.merge(vel_zoom_df, consecutive_streaks_vel_clean,left_on="min_datetime", right_on="min_datetime")

    return merged_vel_df

def post_process_full_data(multi_model_data, model, gate, year=None, start_date=None, end_date=None):
    merged_gate_df = post_process_gateop(multi_model_data, model, gate, year, start_date, end_date)
    merged_vel_df = post_process_velocity(multi_model_data, model, gate, year, start_date, end_date)
    full_merged_df = pd.merge(merged_vel_df, merged_gate_df, left_on="datetime", right_on="datetime")
    full_merged_df['time_unit'] = 0.25

    full_merged_df['gate_status'] = np.where(full_merged_df['gate_status'], "Closed", "Open")
    full_merged_df['week'] = full_merged_df['datetime'].dt.isocalendar().week
    full_merged_df['gate'] = gate
    full_merged_df['model'] = model
    
    return full_merged_df

def calc_avg_daily_vel(post_processed_data: DataFrame) -> DataFrame:
    """
    Calculate daily average of total amount of time velocity is above and below 8ft/s.

    Parameters:
    - post_processed_data (DataFrame): post processed dataframe.

    Returns:
    - DataFrame
    """
    daily_velocity = post_processed_data.groupby(["date", "Velocity_Category"])["time_unit"].sum().reset_index()
    avg_daily_velocity = pd.DataFrame(daily_velocity.groupby("Velocity_Category")['time_unit'].sum()/daily_velocity["date"].nunique()).reset_index()
    
    return avg_daily_velocity

def calc_avg_daily_gate(post_processed_data: DataFrame) -> DataFrame:
    """
    Calculate daily average of total amount of time gate is open and closed.

    Parameters:
    - post_processed_data (DataFrame): post processed dataframe.

    Returns:
    - DataFrame
    """
        
    daily_gate = post_processed_data.groupby(["date","gate_status"])["time_unit"].sum().reset_index()
    avg_daily_gate = pd.DataFrame(daily_gate.groupby("gate_status")['time_unit'].sum()/daily_gate["date"].nunique()).reset_index()

    return avg_daily_gate

def calc_avg_len_consec_vel(post_processed_data: DataFrame) -> DataFrame:
    """
    Calculate daily average of length of consecutive hours velocity is above and below 8ft/s.

    Parameters:
    - post_processed_data (DataFrame): post processed dataframe.

    Returns:
    - DataFrame
    """
    daily_velocity_stats = post_processed_data.groupby(["date", "Velocity_Category"]).agg(
        unique_consecutive_groups=("consecutive_groups", "nunique"),
        total_time=("time_unit", "sum")
    ).reset_index()

    daily_velocity_stats["daily_average_time_per_consecutive_group"] = (
        daily_velocity_stats["total_time"] / daily_velocity_stats["unique_consecutive_groups"]
    )
    daily_average_per_duration_per_velocity_over_period = daily_velocity_stats.groupby(["Velocity_Category"])['daily_average_time_per_consecutive_group'].mean().reset_index()
    
    return daily_average_per_duration_per_velocity_over_period

def calc_avg_len_consec_gate(post_processed_data: DataFrame) -> DataFrame:
    """
    Calculate daily average of length of consecutive hours gate is open or closed.

    Parameters:
    - post_processed_data (DataFrame): post processed dataframe.

    Returns:
    - DataFrame
    """    
    daily_gate_stats = post_processed_data.groupby(["date", "gate_status"]).agg(
        unique_gate_count=("gate_count", "nunique"),
        total_time=("time_unit", "sum")
    ).reset_index()

    daily_gate_stats["daily_average_time_per_consecutive_gate"] = (
        daily_gate_stats["total_time"] / daily_gate_stats["unique_gate_count"]
    )
    daily_average_per_duration_per_gate_over_period = daily_gate_stats.groupby(["gate_status"])['daily_average_time_per_consecutive_gate'].mean().reset_index()
    
    return daily_average_per_duration_per_gate_over_period

def generate_velocity_gate_charts(full_merged_df):
    """
    Generate bar charts for daily velocity and gate status summaries.

    Parameters:
    full_merged_df (pd.DataFrame): DataFrame containing the merged data with the necessary columns.

    Returns:
    alt.Chart: Combined Altair charts for velocity and gate status duration summaries.
    """
    # Compute summary statistics
    summary_stats_vel = (full_merged_df.groupby(["week", "date", "Velocity_Category"]).
            agg(
                total_velocity_duration=("time_unit", "sum")
            )).reset_index()

    summary_stats_dgl = (full_merged_df.groupby(["week", "date", "gate_status"]).
        agg(
            total_gate_duration=("time_unit", "sum")
        )).reset_index()

    # Extract unique gate identifier
    gate = full_merged_df['gate'].unique()[0]
    model = full_merged_df['model'].unique()[0]

    # Define color palette
    color_palette = {
        "Velocity_Category": {"Over 8ft/s": "#a6cee3", "Under 8ft/s": "#1f78b4"},  # Blues
        "gate_status": {"Closed": "#b2df8a", "Open": "#33a02c"}  # Greens
    }

    # Define brush for selection
    brush = alt.selection_interval(encodings=['x'], mark=alt.BrushConfig(stroke="cyan", strokeOpacity=1))

    # Velocity bar chart
    base_vel = alt.Chart(summary_stats_vel, width=800, height=300).mark_bar().encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("total_velocity_duration:Q", title="Hours"),
        color=alt.condition(
            brush,
            alt.Color(
                'Velocity_Category:N',
                title="Velocity Category",
                scale=alt.Scale(
                    domain=color_palette["Velocity_Category"].keys(),
                    range=color_palette["Velocity_Category"].values()
                )
            ),
            alt.value('lightgray')  # Gray for unselected
        ),
        tooltip=["date:T", "Velocity_Category:N", "total_velocity_duration:Q"]
    ).properties(
        title=f"Daily Velocity at {gate} Over/Under 8 ft/s Duration Summary"
    )

    upper_vel = base_vel.mark_bar(width=alt.RelativeBandSize(0.7), stroke='grey', strokeWidth=0.5).encode(
        alt.X('date:T', title="Date").scale(domain=brush)
    )

    lower_vel = base_vel.properties(
        height=90
    ).add_params(brush)

    vel_bar_chart = upper_vel & lower_vel

    # Gate bar chart
    base_gate = alt.Chart(summary_stats_dgl, width=800, height=300).mark_bar().encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("total_gate_duration:Q", title="Hours"),
        color=alt.condition(
            brush,
            alt.Color(
                'gate_status:N',
                title="Gate Status",
                scale=alt.Scale(
                    domain=color_palette["gate_status"].keys(),
                    range=color_palette["gate_status"].values()
                )
            ),
            alt.value('lightgray')  # Gray for unselected
        ),
        tooltip=["date:T", "gate_status:N", "total_gate_duration:Q"]
    ).properties(
        title=f"Daily {gate} Gate Status Duration Summary"
    )

    upper_gate = base_gate.mark_bar(width=alt.RelativeBandSize(0.7), stroke='grey', strokeWidth=0.5).encode(
        alt.X('date:T', title="Date").scale(domain=brush)
    )

    lower_gate = base_gate.properties(
        height=90
    ).add_params(brush)

    gate_bar_chart = upper_gate & lower_gate

    # Combine charts
    combined_bar_charts = alt.vconcat(
        gate_bar_chart,
        vel_bar_chart
    ).resolve_scale(color='independent').properties(title= f"Model: {model}")

    return combined_bar_charts



In [18]:
# start_zoom = '2010-05-01'
# end_zoom = '2024-11-30'
multi_model_data = generate_multi_model_data(
    data,
    ['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss'],
    gatef,
    elev_list,
    flow_list,
    stn_name,
    stn_list)
    # start_zoom,
    # end_zoom)


In [19]:
multi_model_data.keys()

# multi_model_data['FPV1Ma'].keys()
# gate_data = multi_model_data["FPV2Ma"]["OLD"]["gate_operation_data"]
# gate_data["year"] = gate_data["datetime"].dt.year
# print(gate_data.head())        
# gate_data = gate_data[gate_data["year"] == 2017]
# print(gate_data.tail())
# multi_model_data['FPV1Ma']["OLD"]["gate_operation_data"]["year"] = multi_model_data['FPV1Ma']["OLD"]["gate_operation_data"]["datetime"].dt.year
# years = multi_model_data["FPV1Ma"]["OLD"]["gate_operation_data"]["year"].unique().tolist()

# multi_model_data['FPV1Ma']["GLC"]
# fpv2ma_hydro_full_data.to_csv("fpv2ma_hydro_full_data.csv", index=False)

dict_keys(['FPV1Ma', 'FPV2Ma'])

In [529]:
with open("multi_model_data.pkl", "wb") as f:
    pickle.dump(multi_model_data, f)

In [47]:
with open("multi_model_data.pkl", "rb") as f:
    multi_model_data = pickle.load(f)
print(multi_model_data["FPV1Ma"].keys())


dict_keys(['GLC', 'MID', 'OLD'])


In [48]:

glc_full_merged_df = post_process_full_data(multi_model_data, "FPV1Ma", "GLC", year=2016)
# print(glc_full_merged_df.tail())
glc_full_merged_df = glc_full_merged_df.rename(columns={"value": "velocity"})
glc_avg_daily_velocity = calc_avg_daily_vel(glc_full_merged_df)
glc_avg_daily_gate = calc_avg_daily_gate(glc_full_merged_df)
glc_total_daily_velocity = calc_avg_len_consec_vel(glc_full_merged_df)
glc_total_daily_gate = calc_avg_len_consec_gate(glc_full_merged_df)
print(glc_avg_daily_velocity)
old_full_merged_df = post_process_full_data(multi_model_data, "FPV1Ma", "OLD", year=2016)
old_full_merged_df = old_full_merged_df.rename(columns={"value": "velocity"})
old_avg_daily_velocity = calc_avg_daily_vel(old_full_merged_df)
old_avg_daily_gate = calc_avg_daily_gate(old_full_merged_df)
old_total_daily_velocity = calc_avg_len_consec_vel(old_full_merged_df)
old_total_daily_gate = calc_avg_len_consec_gate(old_full_merged_df)

mid_full_merged_df = post_process_full_data(multi_model_data, "FPV1Ma", "MID", year=2016)
mid_full_merged_df = mid_full_merged_df.rename(columns={"value": "velocity"})
mid_avg_daily_velocity = calc_avg_daily_vel(mid_full_merged_df)
mid_avg_daily_gate = calc_avg_daily_gate(mid_full_merged_df)
mid_total_daily_velocity = calc_avg_len_consec_vel(mid_full_merged_df)
mid_total_daily_gate = calc_avg_len_consec_gate(mid_full_merged_df)
# print(mid_full_merged_df)

C:\Users\Inigo\AppData\Local\Temp\ipykernel_10164\1393711358.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gate_up_df["gate_status"] = gate_up_df['value']>=10
C:\Users\Inigo\AppData\Local\Temp\ipykernel_10164\1393711358.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gate_up_df['consecutive_groups'] = (gate_up_df['value'] != gate_up_df['value'].shift()).cumsum()
C:\Users\Inigo\AppData\Local\Temp\ipykernel_10164\1393711358.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

  Velocity_Category  time_unit
0        Over 8ft/s  12.448598
1       Under 8ft/s  11.551402


In [44]:
glc_full_merged_df.dtypes

datetime                datetime64[ns]
velocity                       float64
year                             int32
Velocity_Category               object
consecutive_groups               int64
min_datetime            datetime64[ns]
max_datetime            datetime64[ns]
date                            object
count                            int64
streak_duration                float64
gate_status                     object
gate_min_datetime       datetime64[ns]
gate_max_datetime       datetime64[ns]
gate_count                       int64
gate_streak_duration           float64
time_unit                      float64
week                            UInt32
gate                            object
model                           object
dtype: object

In [49]:
# print(glc_full_merged_df.head())
glc_chart = generate_velocity_gate_charts(glc_full_merged_df)
mid_chart = generate_velocity_gate_charts(mid_full_merged_df)
old_chart = generate_velocity_gate_charts(old_full_merged_df)


In [50]:
alt.hconcat(
        glc_chart,
        mid_chart,
        old_chart
    ).resolve_scale(color='independent')

alt.HConcatChart(...)

In [28]:
summary_stats_vel = (glc_full_merged_df.groupby(["week", "date", "Velocity_Category"]).
        agg(
            total_velocity_duration = ("time_unit", "sum")
        )).reset_index()
summary_stats_dgl = (glc_full_merged_df.groupby(["week", "date", "gate_status"]).
    agg(
        total_gate_duration = ("time_unit", "sum")
    )).reset_index()

# print(summary_stats_vel.tail())
print(summary_stats_dgl.tail(40))
gate = glc_full_merged_df['gate'].unique()[0]
# print(gate)
color_palette = {
    "Velocity_Category": {"Over 8ft/s": "#a6cee3", "Under 8ft/s": "#1f78b4"},  # Blues
    "gate_status": {"Closed": "#b2df8a", "Open": "#33a02c"}  # Greens
}
# print(glc_full_merged_df.head())
brush = alt.selection_interval(encodings=['x'], mark=alt.BrushConfig(stroke="cyan", strokeOpacity=1))
base_vel = alt.Chart(summary_stats_vel, width=800, height=300).mark_bar().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("total_velocity_duration:Q", title="Hours"),
    color=alt.condition(
        brush,
        alt.Color(
            'Velocity_Category:N',
            title="Velocity Category",
            scale=alt.Scale(
                domain=color_palette["Velocity_Category"].keys(),
                range=color_palette["Velocity_Category"].values()
            )
        ),
        alt.value('lightgray')  # Gray for unselected
    ),
    tooltip=["date:T", "Velocity_Category:N", "total_velocity_duration:Q"]
).properties(
    title=f"Daily Velocity at GLC Over/Under 8 ft/s Duration Summary"
)
upper_vel = base_vel.mark_bar(width=alt.RelativeBandSize(0.7),stroke='grey', strokeWidth=0.5).encode(
    alt.X('date:T', title="Date").scale(domain=brush)
)
lower_vel = base_vel.properties(
    height=90
).add_params(brush)
vel_bar_chart = upper_vel & lower_vel

#gate graph
base_gate = alt.Chart(summary_stats_dgl, width=800, height=300).mark_bar().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("total_gate_duration:Q", title="Hours"),
    color=alt.condition(
        brush,
        alt.Color(
            'gate_status:N',
            title="Gate Status",
            scale=alt.Scale(
                domain=color_palette["gate_status"].keys(),
                range=color_palette["gate_status"].values()
            )
        ),
        alt.value('lightgray')  # Gray for unselected
    ),
    tooltip=["date:T", "gate_status:N", "total_gate_duration:Q"]
).properties(
    title=f"Daily {gate} Gate Status Duration Summary"
)

upper_gate = base_gate.mark_bar(width=alt.RelativeBandSize(0.7), stroke='grey', strokeWidth=0.5).encode(
    alt.X('date:T', title="Date").scale(domain=brush)
)
lower_gate = base_gate.properties(
    height=90
).add_params(brush)

gate_bar_chart = upper_gate & lower_gate
combined_bar_charts = alt.vconcat(
    gate_bar_chart,
    vel_bar_chart
    ).resolve_scale(color='independent')

     week        date gate_status  total_gate_duration
899    48  2016-11-29      Closed                 21.5
900    48  2016-11-29        Open                  2.5
901    48  2016-11-30      Closed                 21.0
902    48  2016-11-30        Open                  3.0
903    48  2016-12-01      Closed                 17.5
904    48  2016-12-01        Open                  6.5
905    48  2016-12-02        Open                 24.0
906    48  2016-12-03        Open                 24.0
907    48  2016-12-04        Open                 24.0
908    49  2016-12-05        Open                 24.0
909    49  2016-12-06        Open                 24.0
910    49  2016-12-07        Open                 24.0
911    49  2016-12-08        Open                 24.0
912    49  2016-12-09        Open                 24.0
913    49  2016-12-10        Open                 24.0
914    49  2016-12-11        Open                 24.0
915    50  2016-12-12        Open                 24.0
916    50 

In [31]:
combined_bar_charts

alt.VConcatChart(...)

In [51]:
glc_full_merged_df

,datetime,velocity,year,Velocity_Category,consecutive_groups,min_datetime,max_datetime,date,count,streak_duration,gate_status,gate_min_datetime,gate_max_datetime,gate_count,gate_streak_duration,time_unit,week,gate,model
0,2016-05-01 00:00:00,-0.491982,2016,Under 8ft/s,1,2016-05-01 00:00:00,2016-05-01 04:15:00,2016-05-01,18,4.5,Open,2016-05-01 00:00:00,2016-05-01 03:45:00,16,4.0,0.25,17,GLC,FPV1Ma
1,2016-05-01 00:15:00,-0.554725,2016,Under 8ft/s,1,2016-05-01 00:00:00,2016-05-01 04:15:00,2016-05-01,18,4.5,Open,2016-05-01 00:00:00,2016-05-01 03:45:00,16,4.0,0.25,17,GLC,FPV1Ma
2,2016-05-01 00:30:00,-0.564495,2016,Under 8ft/s,1,2016-05-01 00:00:00,2016-05-01 04:15:00,2016-05-01,18,4.5,Open,2016-05-01 00:00:00,2016-05-01 03:45:00,16,4.0,0.25,17,GLC,FPV1Ma
3,2016-05-01 00:45:00,-0.582945,2016,Under 8ft/s,1,2016-05-01 00:00:00,2016-05-01 04:15:00,2016-05-01,18,4.5,Open,2016-05-01 00:00:00,2016-05-01 03:45:00,16,4.0,0.25,17,GLC,FPV1Ma
4,2016-05-01 01:00:00,-0.580842,2016,Under 8ft/s,1,2016-05-01 00:00:00,2016-05-01 04:15:00,2016-05-01,18,4.5,Open,2016-05-01 00:00:00,2016-05-01 03:45:00,16,4.0,0.25,17,GLC,FPV1Ma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20539,2016-11-30 22:45:00,11.588810,2016,Over 8ft/s,928,2016-11-30 20:00:00,2016-11-30 23:45:00,2016-11-30,16,4.0,Closed,2016-11-30 20:00:00,2016-11-30 23:45:00,16,4.0,0.25,48,GLC,FPV1Ma
20540,2016-11-30 23:00:00,11.704375,2016,Over 8ft/s,928,2016-11-30 20:00:00,2016-11-30 23:45:00,2016-11-30,16,4.0,Closed,2016-11-30 20:00:00,2016-11-30 23:45:00,16,4.0,0.25,48,GLC,FPV1Ma
20541,2016-11-30 23:15:00,11.827288,2016,Over 8ft/s,928,2016-11-30 20:00:00,2016-11-30 23:45:00,2016-11-30,16,4.0,Closed,2016-11-30 20:00:00,2016-11-30 23:45:00,16,4.0,0.25,48,GLC,FPV1Ma
20542,2016-11-30 23:30:00,11.865903,2016,Over 8ft/s,928,2016-11-30 20:00:00,2016-11-30 23:45:00,2016-11-30,16,4.0,Closed,2016-11-30 20:00:00,2016-11-30 23:45:00,16,4.0,0.25,48,GLC,FPV1Ma


In [ ]:

color_palette = {
    "Velocity_Category": {"Over 8ft/s": "#a6cee3", "Under 8ft/s": "#1f78b4"},  # Blues
    "gate_status": {"Closed": "#b2df8a", "Open": "#33a02c"}  # Greens
}
interval = alt.selection_interval(encodings=['x'],
                                  mark=alt.BrushConfig(fill='blue')
                                  )
base = alt.Chart(glc_full_merged_df).mark_line(color=color_palette["Velocity_Category"]["Under 8ft/s"]).encode(
    x=alt.X(
        'yearmonthdatehoursminutes(datetime):T', 
        title='Datetime', 
        axis=alt.Axis(format='%b %d, %Y', labelAngle=-45, title='Date')
    ),
    y=alt.Y('velocity:Q', title='Velocity (ft/s)'),
).add_params(interval).properties(
    title="Fish Passage Velocity and Gate Status Zoomed",
    height=300
)
closed_gates = glc_full_merged_df[['gate_min_datetime', 'gate_max_datetime', 'gate_status']].drop_duplicates().reset_index(drop=True)
area_dgl_true = alt.Chart(closed_gates).mark_rect(
    color='orange'
).encode(
    x='gate_min_datetime:T',
    x2='gate_max_datetime:T',
    opacity=alt.value(0.2),
    color=alt.value(color_palette["gate_status"]["Closed"]),
    # color=alt.condition(interval, alt.value('orange'), alt.value('lightgray'))
).transform_filter(
    alt.datum.gate_status == "Closed"
)
yrule = alt.Chart(glc_full_merged_df).mark_rule(color = "red", strokeDash=[12, 6], size=1.5).encode(
        y=alt.datum(8)
).properties(
    width=300,
    height=300
)
nearest = alt.selection_point(nearest=True, on="pointerover",
                              fields=["datetime"], empty=False)
points = base.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
# # Draw a rule at the location of the selection
rules = alt.Chart(glc_full_merged_df).transform_calculate(
    FlowVelocityDuration = "'Flow ' + datum.Velocity_Category + ' duration is ' + datum.streak_duration + ' hours'",
    GateStatusDuration = "'Gate ' + datum.gate_status + ' duration is ' + datum.gate_streak_duration + ' hours'"
).mark_rule(color="gray").encode(
    x="datetime:T",
    opacity=alt.condition(nearest, alt.value(0.3), alt.value(0)),
    tooltip=[alt.Tooltip('yearmonthdatehoursminutes(datetime):T', title='Datetime'),
             alt.Tooltip('velocity:Q', title= "Velocity (ft/s)", format=".2f"),
             alt.Tooltip('FlowVelocityDuration:N', title="Flow Velocity Duration"),
             alt.Tooltip('GateStatusDuration:N', title="Gate Status Duration")
             ],
).add_params(nearest)
vel_text = alt.Chart(glc_full_merged_df).mark_text(align='right').encode(
    y=alt.Y('stat:N', axis=None),
    text=alt.Text('summary:N')
).transform_filter(
    interval
).transform_aggregate(
    max_velocity='max(velocity)',  
    min_velocity='min(velocity)',
    avg_velocity='mean(velocity)'
).transform_fold(
    ['max_velocity', 'min_velocity', 'avg_velocity'],  # Separate each statistic
    as_=['stat', 'value']
).transform_calculate(
    summary='datum.stat + ": " + format(datum.velocity, ".2f")'
)
velocity = vel_text.encode(text='summary:N').properties(
    title=alt.Title(text='Flow Velocity Summary', align='center')
)
vel_duration_text = alt.Chart(glc_full_merged_df).mark_text(align='right').encode(
    y=alt.Y('Velocity_Category:N', axis=None),
    text=alt.Text('summary:N')
).transform_filter(
    interval
).transform_aggregate(
    total_time='sum(time_unit)',
    groupby=["Velocity_Category"]
).transform_fold(
    ['total_time'],  # Separate each statistic
    as_=['stat', 'value']
).transform_calculate(
    summary='datum.Velocity_Category + ": " + format(datum.total_time, ".2f") + " hours"'
)
velocity_duration = vel_duration_text.encode(text='summary:N').properties(
    title=alt.Title(text='Velocity Duration Summary', align='center')
)
gate_duration_text = alt.Chart(glc_full_merged_df).mark_text(align='right').encode(
    y=alt.Y('gate_status:N', axis=None),
    text=alt.Text('summary:N')
).transform_filter(
    interval
).transform_aggregate(
    total_time='sum(time_unit)',
    groupby=["gate_status"]
).transform_fold(
    ['total_time'],  # Separate each statistic
    as_=['stat', 'value']
).transform_calculate(
    summary='datum.gate_status + ": " + format(datum.total_time, ".2f") + " hours"'
)
gate_duration = gate_duration_text.encode(text='summary:N').properties(
    title=alt.Title(text='Gate Status Duration Summary', align='center')
)
layered_chart = alt.layer(base, points, yrule, rules, area_dgl_true).properties(
    width=700,  
    height=400
)
velocity = velocity.properties(width=200, height=100)
# daily_velocity = daily_velocity.properties(width=200, height=100)
velocity_duration = velocity_duration.properties(width=200, height=100)
gate_duration = gate_duration.properties(width=200, height=100)
text_summary = alt.vconcat(velocity, velocity_duration, gate_duration)

layered_chart
combined_chart = alt.hconcat(
    layered_chart,
    # velocity
    text_summary
)
    


In [33]:

combined_chart

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.HConcatChart(...)

In [303]:
unique_dates = full_merged_df['date'].unique()
gate = full_merged_df['gate'].unique()[0]

avg_line_velocity = alt.Chart(avg_daily_velocity).mark_rule(strokeWidth=2, color='red', strokeDash=[5, 5]).encode(
    y=alt.Y('time_unit:Q'),
    detail='Velocity_Category:N',
    tooltip=alt.Tooltip('time_unit:Q',  format=".2f")  
)
avg_line_gate = alt.Chart(avg_daily_gate).mark_rule(strokeWidth=4, color='red', strokeDash=[5, 5]).encode(
    y=alt.Y('time_unit:Q'),
    detail='gate_status:N',
    # alt.Tooltip('GLC:Q', title= "Velocity (ft/s)", format=".2f")
    tooltip=alt.Tooltip('time_unit:Q',  format=".2f")  # Static tooltip for the rule line
    ).interactive()

summary_stats_vel = (full_merged_df.groupby(["week", "date", "Velocity_Category"]).
    agg(
        total_velocity_duration = ("time_unit", "sum")
    )).reset_index()
# print(summary_stats_vel.head())
summary_stats_gate_status = (full_merged_df.groupby(["week", "date", "gate_status"]).
    agg(
        total_gate_duration = ("time_unit", "sum")
    )).reset_index()
print(summary_stats_gate_status.head())

summary_stats_combo = (full_merged_df.groupby(["week", "date", "gate_status", "Velocity_Category"]).
    agg(
        total_duration = ("time_unit", "sum")
    )).reset_index()
print(summary_stats_combo.head())

unique_weeks = list(full_merged_df.week.unique())
week_dropdown = alt.binding_select(options=unique_weeks, name="Select Week:")
week_selection = alt.selection_single(fields=["week"], bind=week_dropdown, empty = "none")


brush = alt.selection_interval(encodings=['x'], mark=alt.BrushConfig(stroke="cyan", strokeOpacity=1))

# Create the bar graph
base_vel = alt.Chart(summary_stats_vel, width=400, height=400).mark_bar(color="green").encode(
        x=alt.X("date:T", title="Velocity Category"),
        y=alt.Y("total_velocity_duration:Q", title="Hours"),
        # color="Velocity_Category:N",
        color=alt.condition(brush, 'Velocity_Category:N', alt.value('lightgray')),
        # column="Velocity_Category:N",
        tooltip=["date:T", "Velocity_Category:N", "total_velocity_duration:Q"],
# ).add_selection(week_selection
).properties(
    title="Daily Velocity Over/Under 8 ft/s Duration Summary"
)
upper_vel = base_vel.mark_bar(width=alt.RelativeBandSize(0.7)).encode(
    alt.X('date:T').scale(domain=brush)
)
lower_vel = base_vel.properties(
    height=90
).add_params(brush)
vel_bar_chart = upper_vel & lower_vel
# vel_chart_with_avg = vel_bar_chart + avg_line_velocity
base_gate = alt.Chart(summary_stats_gate_status, width=400, height=400).mark_bar(
    color="steelblue",
    # width=alt.RelativeBandSize(0.7)
).encode(
    x=alt.X("date:T", title="Gate Status"), 
    y=alt.Y("total_gate_duration:Q", title="Hours"),
    color=alt.condition(brush, 'gate_status:N', alt.value('lightgray')),
    tooltip=["date:T","gate_status:N", "total_gate_duration:Q"]
).properties(
    title="Daily Gate Status Duration Summary"
)
# gate_bar_chart = alt.Chart(summary_stats_dgl).mark_bar(color="steelblue").encode(
#         x=alt.X("date", title="Gate Status",axis=alt.Axis()),
#         y=alt.Y("total_gate_duration:Q", title="Hours"),
#         color="DGL:N",
#         # column="DGL:N",
#         tooltip=["DGL:N", "total_gate_duration:Q"],
# # ).add_selection(week_selection
# ).properties(
#     title="Weekly Gate Status Duration Summary", width=200, height=400
# )

upper_gate = base_gate.mark_bar(width=alt.RelativeBandSize(0.7), stroke='grey', strokeWidth=0.5).encode(
    alt.X('date:T').scale(domain=brush)
)
lower_gate = base_gate.properties(
    height=90
).add_params(brush)

gate_bar_chart = upper_gate & lower_gate
# gate_chart_with_avg = gate_bar_chart + avg_line_gate
combined_bar_charts = alt.hconcat(
    gate_bar_chart,
    vel_bar_chart 
)


# combined_bar_charts

base_combo = alt.Chart(summary_stats_combo, width=400, height=400).mark_bar(
    color="steelblue",
    # width=alt.RelativeBandSize(0.7)
).transform_calculate(
    category="datum.gate_status + '-' + datum.Velocity_Category"
).encode(
    x=alt.X("date:T", title="Gate Status"), 
    y=alt.Y("total_duration:Q", title="Hours"),
    color=alt.condition(brush, 'category:N', alt.value('lightgray')),
    tooltip=["date:T","gate_status:N", "total_duration:Q", "Velocity_Category:N"]
).properties(
    title=f"Daily {gate}, Flow Combination Status Duration Summary"
)
upper_combo = base_combo.mark_bar(width=alt.RelativeBandSize(0.7), stroke='grey', strokeWidth=0.5).encode(
    alt.X('date:T').scale(domain=brush)
)
lower_combo = base_combo.properties(
    height=90
).add_params(brush)
combo_bar_chart = upper_combo & lower_combo
combo_bar_chart
#create weekly time series with bars
#eventually will have a 100 year time series
#whats the day thats the most above 8/fts
#look at daily variability/similarity over the entire summer (stacked bar) 
#gate open = low flow
#gate close = high flow (water pushed to passage)

   week        date gate_status  total_gate_duration
0    17  2016-05-01      Closed                18.50
1    17  2016-05-01        Open                 5.50
2    18  2016-05-02      Closed                21.75
3    18  2016-05-02        Open                 2.25
4    18  2016-05-03      Closed                19.75
   week        date gate_status Velocity_Category  total_duration
0    17  2016-05-01      Closed        Over 8ft/s           12.50
1    17  2016-05-01      Closed       Under 8ft/s            6.00
2    17  2016-05-01        Open       Under 8ft/s            5.50
3    18  2016-05-02      Closed        Over 8ft/s           13.25
4    18  2016-05-02      Closed       Under 8ft/s            8.50


C:\Users\Inigo\AppData\Local\Temp\ipykernel_53388\1191255207.py:35: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  week_selection = alt.selection_single(fields=["week"], bind=week_dropdown, empty = "none")


alt.VConcatChart(...)

In [249]:
# gate_up = post_process_gateop(fpv2ma_hydro_full_data, "MID")
vel_zoom_df =fpv2ma_hydro_full_data["GLC"]["vel"]
# vel_zoom_df['datetime'] = vel_zoom_df.index
# vel_zoom_df = vel_zoom_df.reset_index(drop=True) 
# print(gate_up.head()) # fpv2ma_hydro_full_data['GLC']['gate_operation_data']
vel_zoom_df['Velocity_Category'] = np.where(vel_zoom_df['value'] >= 8, "Over 8ft/s", "Under 8ft/s")
#.shift shift value down and compare each value with the previous row; increase value when rows are different
vel_zoom_df['consecutive_groups'] = (vel_zoom_df['Velocity_Category'] != vel_zoom_df['Velocity_Category'].shift()).cumsum()
# print(vel_zoom_df)
vel_zoom_df['min_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('min')
vel_zoom_df['max_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('max')
vel_zoom_df['date'] = vel_zoom_df['datetime'].dt.date.astype(str)
consecutive_streaks_vel = vel_zoom_df.groupby(['consecutive_groups', 'Velocity_Category', 'min_datetime', 'max_datetime']).size().reset_index(name='count')
consecutive_streaks_vel['streak_duration'] = consecutive_streaks_vel['count'] * 15 / 60

consecutive_streaks_vel_clean = consecutive_streaks_vel.drop(['consecutive_groups', 'Velocity_Category', 'max_datetime'], axis=1)
merged_df = pd.merge(vel_zoom_df, consecutive_streaks_vel_clean,left_on="min_datetime", right_on="min_datetime")
# print(merged_df.head(5))
# over_streaks = consecutive_streaks_vel[consecutive_streaks_vel['Velocity_Category'] == 'Over 8ft/s']
# under_streaks = consecutive_streaks_vel[consecutive_streaks_vel['Velocity_Category'] == 'Under 8ft/s']

# # Total duration for over and under 8ft/s
# total_over_duration = over_streaks['count'].sum()
# total_under_duration = under_streaks['count'].sum()

# over_streak_duration =  over_streaks['count'] * 15 / 60
# under_streak_duration = under_streaks['count'] * 15 / 60

# # Group by the Velocity Category and count the number of days in each category
# vel_count_df = vel_zoom_df.groupby('Velocity_Category').size().reset_index(name='count')

# total_time_df = pd.DataFrame({
#     'Velocity_Category': ['Over 8ft/s', 'Under 8ft/s'],
#     'Total Hours': [total_over_duration * 15 / 60, total_under_duration * 15 / 60]
# })

# consecutive_streaks_vel.rename(columns={
#     "Velocity_Category": "Status"},
#     inplace = True)

In [253]:

print(consecutive_streaks_vel_clean.head())

         min_datetime  count  streak_duration
0 2016-05-01 00:00:00     21             5.25
1 2016-05-01 05:15:00     19             4.75
2 2016-05-01 10:00:00     22             5.50
3 2016-05-01 15:30:00     31             7.75
4 2016-05-01 23:15:00     24             6.00


In [ ]:
sdg = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['sdg']
sdg_stage = sdg[sdg['gate_op'].isin(elev_list)]
sdg_flow = sdg[sdg['gate_op'].isin(flow_list)]
sdg_gateop = sdg[sdg['gate_op'].isin(stn_list)]
gate_names = {'MID_GATEOP':'MHO',
              'GLC_GATEOP':'DGL',
              'OLD_GATEOP':'OLD'}
sdg_gateop['gate_op'] = sdg_gateop['gate_op'].replace(gate_names)
print(sdg_gateop.head())
year = "2012"

# sdg_gateop =filter_date(sdg_gateop, start_zoom, end_zoom)
print(sdg_gateop.head())
hydro = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['hydro']
hydro_wl = hydro[hydro['parameter']=="STAGE"]
hydro_wl = hydro_wl[hydro_wl['gate'].isin(stn_name)]
# print(hydro_wl.head())

# print(wl.head())
# print(data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss'].keys())
# # hydro_gateop.rename(columns={'MID_GATEOP':'MHO','GLC_GATEOP':'DGL','OLD_GATEOP':'OLD'}, inplace=True) # renaming to make it easier later.
# print(hydro_gateop['gate'].unique())
# print(hydro_gateop[hydro_gateop['gate']=="DGL"])

C:\Users\Inigo\AppData\Local\Temp\ipykernel_53388\599939783.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdg_gateop['gate_op'] = sdg_gateop['gate_op'].replace(gate_names)


             datetime  value  unit gate_op scenario
0 2015-12-31 00:15:00    NaN  FEET     DGL   FPV1MA
1 2015-12-31 00:30:00    NaN  FEET     DGL   FPV1MA
2 2015-12-31 00:45:00    NaN  FEET     DGL   FPV1MA
3 2015-12-31 01:00:00    NaN  FEET     DGL   FPV1MA
4 2015-12-31 01:15:00    NaN  FEET     DGL   FPV1MA
                 datetime     value  unit        gate_op scenario
11711 2016-05-01 00:00:00  3.680600  FEET  GLC_GATE_DOWN   FPV1MA
11712 2016-05-01 00:15:00  3.818419  FEET  GLC_GATE_DOWN   FPV1MA
11713 2016-05-01 00:30:00  3.946771  FEET  GLC_GATE_DOWN   FPV1MA
11714 2016-05-01 00:45:00  4.091312  FEET  GLC_GATE_DOWN   FPV1MA
11715 2016-05-01 01:00:00  4.218915  FEET  GLC_GATE_DOWN   FPV1MA


In [208]:
full_data = {}

for i, gate_name in enumerate(gatef['name']):

    flow_data = sdg_flow[sdg_flow.gate_op == gatef["flow_op"][i]]
    gate_data = sdg_stage[sdg_stage.gate_op== gatef['gate_status'][i]]
     
    # Drop rows with NaN values from flow_data and gate_data
    flow_data = flow_data.dropna()
    gate_data = gate_data.dropna()
    
    full_data[gatef["ID"][i]] = {
        'name': gatef['name'][i],
        'bottom_elev': gatef['bottom_elev'][i],
        'width': gatef['width'][i],
        'flow_data': flow_data,
        'gate_data': gate_data
    }
sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']
sdg_flow_GLC_GATE_UP = full_data['GLC']['gate_data']
glc_bottom_elev = full_data['GLC']['bottom_elev']
glc_width = full_data['GLC']['width']



In [196]:
sdg_flow_MID_FLOW_FISH = full_data['MID']['flow_data']['value']
sdg_flow_MID_GATE_UP = full_data['MID']['gate_data']['value']
mid_bottom_elev = full_data['MID']['bottom_elev']
mid_width = full_data['MID']['width']

sdg_flow_OLD_FLOW_FISH = full_data['OLD']['flow_data']['value']
sdg_flow_OLD_GATE_UP = full_data['OLD']['gate_data']['value']
old_bottom_elev = full_data['OLD']['bottom_elev']
old_width = full_data['OLD']['width']

In [209]:
thr_vel = 8
# mask = (sdg_flow.index.month>=5) & (sdg_flow.index.month<12) # time when gate are in operation
# vel=pd.DataFrame()
sdg_flow_GLC_FLOW_FISH.set_index('datetime', inplace=True)
sdg_flow_GLC_FLOW_FISH = sdg_flow_GLC_FLOW_FISH['value']
# print(sdg_flow_GLC_FLOW_FISH.head())
# sdg_flow_GLC_GATE_UP.set_index('datetime', inplace=True)
# sdg_flow_GLC_GATE_UP = sdg_flow_GLC_GATE_UP['value']
# sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']['value']
vel = pd.DataFrame()
full_data["GLC"]['vel'] = calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width)
# full_data["MID"]['vel'] = calc_vel(sdg_flow_MID_FLOW_FISH,sdg_flow_MID_GATE_UP,mid_bottom_elev, mid_width)
# full_data["OLD"]['vel'] = calc_vel(sdg_flow_OLD_FLOW_FISH,sdg_flow_OLD_GATE_UP,old_bottom_elev, old_width)

# vel['MID'] = calc_vel(sdg_flow.MID_FLOW_FISH,sdg_flow.MID_GATE_UP,gatef['bottom_elev'][1], gatef['width'][1] )
# vel['OLD'] = calc_vel(sdg_flow.OLD_FLOW_FISH,sdg_flow.OLD_GATE_UP,gatef['bottom_elev'][2], gatef['width'][2] )
# print(full_data["GLC"]['vel'].tail())
print(full_data.keys())

TypeError: Addition/subtraction of integers and integer-arrays with DatetimeArray is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`